<a href="https://colab.research.google.com/github/Rohith-Bollam/new-project/blob/main/Instagram_Post_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
pip install langgraph langchain

In [83]:
!pip install langchain-google-genai

  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [84]:
from google.colab import userdata
API_KEY=userdata.get('GOOGLE_API_KEY_1')

In [87]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=API_KEY)

In [88]:
from langgraph.graph import MessageGraph,END
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate

In [90]:
writer_prompt = ChatPromptTemplate.from_template(
    "Write a Instagram post about this topic: {topic}"
)

writer_chain = writer_prompt | model

reviewer_prompt = ChatPromptTemplate.from_template(
    "Review the following Instagram post and suggest improvements: {post}"
)

reviewer_chain = reviewer_prompt | model

In [91]:
graph = MessageGraph()

def writer_node(state):
  print(state)
  topic = state[-1].content
  print(topic)
  post = writer_chain.invoke({"topic": topic})
  print(type(post))
  return state + [AIMessage(content = post.content)]

def reviewer_node(state):
  print(state)
  post = state[-1].content
  review = reviewer_chain.invoke({"post": post})
  return state + [AIMessage(content = review.content)]

graph.add_node("Writer", writer_node)
graph.add_node("Reviewer", reviewer_node)
graph.set_entry_point("Writer")

def condition_check(state):
  print("From condition_check")
  if len(state)>=5:
    print("Printing from If")
    return END
  else:
    print("Printing from Else")
    return "Reviewer"

graph.add_conditional_edges("Writer", condition_check)
graph.add_edge("Reviewer", "Writer")

app = graph.compile()

In [92]:
response = app.invoke([HumanMessage(content = "Job market in India?")])
for message in response:
  print(message.content)

[HumanMessage(content='Job market in India?', additional_kwargs={}, response_metadata={}, id='d625e59e-c3da-4bb1-831b-8fb62a1c491a')]
Job market in India?
<class 'langchain_core.messages.ai.AIMessage'>
From condition_check
Printing from Else
[HumanMessage(content='Job market in India?', additional_kwargs={}, response_metadata={}, id='d625e59e-c3da-4bb1-831b-8fb62a1c491a'), AIMessage(content="## Navigating the Indian Job Market: Opportunities & Challenges 🇮🇳💼\n\nThe Indian job market is a dynamic landscape, constantly evolving with new trends and industries emerging. Whether you're a seasoned professional or a fresh graduate, understanding the current climate is crucial for career success.\n\n**Here's a quick snapshot:**\n\n* **High Growth Sectors:** Tech (AI, Data Science, Cloud Computing), Healthcare, Renewable Energy, and E-commerce are booming, creating a surge in demand for skilled professionals. 🚀\n* **Skill Gap:** While there are opportunities, a significant skill gap exists. Ups